<h2><center> COMS 573 Machine Learning </center>
<center> Lab 1 </center>
  <center> Naive Bayes Classifier for Text Classification </center></h2>
  
 <h3><center>Ramakrishnan</center> 
 <center> ramkris@iastate.edu </center></h3>

# Prepreocess Data and Import Libraries

In [1]:
import pandas as pd
import numpy as np
import math
from collections import defaultdict

In [2]:
#Read Training Data 

train_data = pd.read_csv('train_data.csv', names = ['docIdx', 'wordIdx', 'count'])
train_data.index = train_data.index+1
train_data


,docIdx,wordIdx,count
1,1,1,4
2,1,2,2
3,1,3,10
4,1,4,4
5,1,5,2
...,...,...,...
1467341,11269,47387,1
1467342,11269,48339,1
1467343,11269,48919,1
1467344,11269,51544,1


In [3]:
#Read Training Label
train_label = pd.read_csv('train_label.csv', names = ['labelId'])
train_label.index = train_label.index+1
train_label

,labelId
1,1
2,1
3,1
4,1
5,1
...,...
11265,20
11266,20
11267,20
11268,20


In [4]:
vocab = pd.read_csv("vocabulary.txt", names = ['token'])
vocab_length = vocab.shape[0]
vocab.index = vocab.index+1
vocab

,token
1,archive
2,name
3,atheism
4,resources
5,alt
...,...
61184,aeroplane
61185,gosple
61186,ephas
61187,kltensme


In [5]:
labelVal = train_label['labelId'].to_numpy()
labelVal = np.insert(labelVal, 0, 0, axis=0)
print(labelVal)

[ 0  1  1 ... 20 20 20]


In [6]:
docVal = train_data['docIdx'].to_numpy()
print(docVal)

[    1     1     1 ... 11269 11269 11269]


In [7]:
#Add category omega to the Dataframe
omegaVal = []
for item in docVal:
    omegaVal.append(int(labelVal[item]))

train_data['omega'] = pd.Series(omegaVal, index = train_data.index)

train_data.head()

,docIdx,wordIdx,count,omega
1,1,1,4,1
2,1,2,2,1
3,1,3,10,1
4,1,4,4,1
5,1,5,2,1


# 2.1 Learn the Naive Bayes Model

### Calculate Class Prior Probabilities

In [8]:
# Calculate class Prior Probabilities
examples = train_label.labelId.count()

prior = [0]*21
for j in range(1,21):
    docs = (train_label.loc[train_label['labelId'] == j]).labelId.count()
    prior_val = docs/examples
    prior[j] = prior_val
    print("P(omega = "+ str(j) + ") : " + str(prior_val))


P(omega = 1) : 0.04259472890229834
P(omega = 2) : 0.05155736977549028
P(omega = 3) : 0.05075871860857219
P(omega = 4) : 0.05208980388676901
P(omega = 5) : 0.051024935664211554
P(omega = 6) : 0.052533498979501284
P(omega = 7) : 0.051646108794036735
P(omega = 8) : 0.052533498979501284
P(omega = 9) : 0.052888455053687104
P(omega = 10) : 0.0527109770165942
P(omega = 11) : 0.05306593309078002
P(omega = 12) : 0.0527109770165942
P(omega = 13) : 0.05244475996095483
P(omega = 14) : 0.0527109770165942
P(omega = 15) : 0.052622237998047744
P(omega = 16) : 0.05315467210932647
P(omega = 17) : 0.04836276510781791
P(omega = 18) : 0.05004880646020055
P(omega = 19) : 0.04117490460555506
P(omega = 20) : 0.033365870973467035


In [9]:
vocab['wordIdx'] = vocab.index
vocab

,token,wordIdx
1,archive,1
2,name,2
3,atheism,3
4,resources,4
5,alt,5
...,...,...
61184,aeroplane,61184
61185,gosple,61185
61186,ephas,61186
61187,kltensme,61187


In [10]:
train_data
mixed = pd.merge(vocab, train_data, on='wordIdx', how = 'outer')
mixed = mixed.fillna(0)
mixed


,token,wordIdx,docIdx,count,omega
0,archive,1,1.0,4.0,1.0
1,archive,1,47.0,2.0,1.0
2,archive,1,196.0,3.0,1.0
3,archive,1,432.0,2.0,1.0
4,archive,1,433.0,2.0,1.0
...,...,...,...,...,...
1474553,aeroplane,61184,0.0,0.0,0.0
1474554,gosple,61185,0.0,0.0,0.0
1474555,ephas,61186,0.0,0.0,0.0
1474556,kltensme,61187,0.0,0.0,0.0


### Calculate n<sub>k<sub>

In [11]:
#Calculate nk
nk_df = mixed.groupby(['wordIdx','omega'])
nk = nk_df['count'].sum()
nk = nk.unstack()
nk
nk = nk.fillna(0)    
nk

omega,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0
wordIdx,,,,,,,,,,,,,,,,,,,,,
1,0.0,13.0,60.0,11.0,8.0,6.0,47.0,0.0,9.0,14.0,...,1.0,52.0,3.0,15.0,48.0,0.0,19.0,10.0,0.0,0.0
2,0.0,63.0,59.0,69.0,31.0,33.0,222.0,28.0,54.0,67.0,...,67.0,90.0,33.0,39.0,82.0,123.0,33.0,154.0,39.0,45.0
3,0.0,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,9.0
4,0.0,9.0,17.0,17.0,0.0,1.0,79.0,2.0,0.0,4.0,...,0.0,11.0,2.0,13.0,22.0,7.0,6.0,9.0,23.0,2.0
5,0.0,82.0,14.0,21.0,10.0,1.0,15.0,2.0,13.0,4.0,...,1.0,59.0,5.0,20.0,12.0,14.0,11.0,2.0,17.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
nk = nk.drop(nk.columns[0], axis=1)

In [13]:
nk

omega,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0
wordIdx,,,,,,,,,,,,,,,,,,,,
1,13.0,60.0,11.0,8.0,6.0,47.0,0.0,9.0,14.0,1.0,1.0,52.0,3.0,15.0,48.0,0.0,19.0,10.0,0.0,0.0
2,63.0,59.0,69.0,31.0,33.0,222.0,28.0,54.0,67.0,33.0,67.0,90.0,33.0,39.0,82.0,123.0,33.0,154.0,39.0,45.0
3,275.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,9.0
4,9.0,17.0,17.0,0.0,1.0,79.0,2.0,0.0,4.0,2.0,0.0,11.0,2.0,13.0,22.0,7.0,6.0,9.0,23.0,2.0
5,82.0,14.0,21.0,10.0,1.0,15.0,2.0,13.0,4.0,1.0,1.0,59.0,5.0,20.0,12.0,14.0,11.0,2.0,17.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Calculate n

In [14]:
#Calculate n
n_df = mixed.groupby('omega')
n = n_df['count'].sum()
n = n.drop(n.index[0])
n

omega
1.0     148812.0
2.0     110358.0
3.0      90767.0
4.0      99146.0
5.0      86190.0
6.0     152846.0
7.0      61094.0
8.0     114102.0
9.0     102631.0
10.0    107898.0
11.0    141267.0
12.0    200456.0
13.0    103173.0
14.0    155338.0
15.0    153714.0
16.0    201267.0
17.0    175914.0
18.0    254805.0
19.0    186426.0
20.0    119096.0
Name: count, dtype: float64

### Calculate P<sub>MLE</sub>

In [15]:
#Calculabe P_MLE
p_mle = nk.divide(n)
p_mle

omega,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0
wordIdx,,,,,,,,,,,,,,,,,,,,
1,0.000087,0.000544,0.000121,0.000081,0.000070,0.000307,0.000000,0.000079,0.000136,0.000009,0.000007,0.000259,0.000029,0.000097,0.000312,0.000000,0.000108,0.000039,0.000000,0.000000
2,0.000423,0.000535,0.000760,0.000313,0.000383,0.001452,0.000458,0.000473,0.000653,0.000306,0.000474,0.000449,0.000320,0.000251,0.000533,0.000611,0.000188,0.000604,0.000209,0.000378
3,0.001848,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000079,0.000000,0.000000,0.000000,0.000076
4,0.000060,0.000154,0.000187,0.000000,0.000012,0.000517,0.000033,0.000000,0.000039,0.000019,0.000000,0.000055,0.000019,0.000084,0.000143,0.000035,0.000034,0.000035,0.000123,0.000017
5,0.000551,0.000127,0.000231,0.000101,0.000012,0.000098,0.000033,0.000114,0.000039,0.000009,0.000007,0.000294,0.000048,0.000129,0.000078,0.000070,0.000063,0.000008,0.000091,0.000193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61184,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
61185,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
61186,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Calculate P<sub>BE</sub>

In [16]:
#Calculate P_BE
nk_new = nk+1
p_be = (nk_new).divide(n+vocab_length)
p_be


omega,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0
wordIdx,,,,,,,,,,,,,,,,,,,,
1,0.000067,0.000356,0.000079,0.000056,0.000047,0.000224,0.000008,0.000057,0.000092,0.000012,0.000010,0.000203,0.000024,0.000074,0.000228,0.000004,0.000084,0.000035,0.000004,0.000006
2,0.000305,0.000350,0.000461,0.000200,0.000231,0.001042,0.000237,0.000314,0.000415,0.000201,0.000336,0.000348,0.000207,0.000185,0.000386,0.000472,0.000143,0.000491,0.000162,0.000255
3,0.001314,0.000006,0.000007,0.000006,0.000007,0.000005,0.000008,0.000006,0.000006,0.000006,0.000005,0.000004,0.000006,0.000005,0.000005,0.000065,0.000004,0.000003,0.000004,0.000055
4,0.000048,0.000105,0.000118,0.000006,0.000014,0.000374,0.000025,0.000006,0.000031,0.000018,0.000005,0.000046,0.000018,0.000065,0.000107,0.000030,0.000030,0.000032,0.000097,0.000017
5,0.000395,0.000087,0.000145,0.000069,0.000014,0.000075,0.000025,0.000080,0.000031,0.000012,0.000010,0.000229,0.000037,0.000097,0.000060,0.000057,0.000051,0.000009,0.000073,0.000133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61184,0.000005,0.000006,0.000007,0.000006,0.000007,0.000005,0.000008,0.000006,0.000006,0.000006,0.000005,0.000004,0.000006,0.000005,0.000005,0.000004,0.000004,0.000003,0.000004,0.000006
61185,0.000005,0.000006,0.000007,0.000006,0.000007,0.000005,0.000008,0.000006,0.000006,0.000006,0.000005,0.000004,0.000006,0.000005,0.000005,0.000004,0.000004,0.000003,0.000004,0.000006
61186,0.000005,0.000006,0.000007,0.000006,0.000007,0.000005,0.000008,0.000006,0.000006,0.000006,0.000005,0.000004,0.000006,0.000005,0.000005,0.000004,0.000004,0.000003,0.000004,0.000006


# Observation:
### From the above values of P<sub>MLE</sub> and P<sub>BE</sub> we observe that P<sub>MLE</sub>  has lots of zeros whereas P<sub>BE</sub> does not have any.
### This is primarily because the numerator has n<sub>k</sub> instead of n<sub>k</sub>+1 which can be zero unlike the latter.

# 2.2 Evaluate the Classifier

In [17]:
def PredictMLE(df):  
    ndoc = len(df['docIdx'])
    df_dict = df.to_dict()
    countMap = defaultdict(dict)
    for i in range(1,ndoc+1):
        docIdx = df_dict['docIdx'][i]
        wordIdx = df_dict['wordIdx'][i]
        count = df_dict['count'][i]
        countMap[docIdx][wordIdx] = count

    countMap = dict(countMap)
    pMap = p_mle.to_dict()
    predict_list = []
    inc = 0

    for docIdx in range(1,len(countMap)+1):
        predict_val = []
        for omega_j in range(1,21):
            val = 0
            for wordIdx in countMap[docIdx]:
                probability = pMap[omega_j][wordIdx]
                product = 0
                #MLE: Handle 0 case by ignoring item
                if probability ==0:
                    val = -math.inf
                    continue
                if probability != 0:
                    Nk = countMap[docIdx][wordIdx]
                    product = np.log(probability)*Nk
                val = val + product 
            class_probability = (np.log(prior[omega_j]))
            predict_val.append(val+class_probability)
        final_val = np.argmax(predict_val)+1
        predict_list.append(final_val)  
    return predict_list


def PredictBE(df):  
    ndoc = len(df['docIdx'])
    df_dict = df.to_dict()
    countMap = defaultdict(dict)
    for i in range(1,ndoc+1):
        docIdx = df_dict['docIdx'][i]
        wordIdx = df_dict['wordIdx'][i]
        count = df_dict['count'][i]
        countMap[docIdx][wordIdx] = count

    countMap = dict(countMap)
    pMap = p_be.to_dict()
    predict_list = []
    inc = 0

    for docIdx in range(1,len(countMap)+1):
        predict_val = []
        for omega_j in range(1,21):
            val = 0 
            for wordIdx in countMap[docIdx]:
                probability = pMap[omega_j][wordIdx]
                logp = np.log(probability)
                prev = val
                Nk = countMap[docIdx][wordIdx]
                product = Nk*logp
                val = val + product 
            class_probability = np.log(prior[omega_j])
            predict_val.append(val+class_probability)
        final_val = np.argmax(predict_val)+1
        predict_list.append(final_val)         
    return predict_list

def CalcAccuracy(predict_list,file):
#     Read training Label
    df = pd.read_csv(file, names = ['labelId'])
    df.index = df.index+1
    df['predicted'] = pd.Series(predict_list, index = df.index)
    match = df[df['labelId'] == df['predicted']]
    predicted_correct = match.shape[0]
    return (predicted_correct/df.shape[0])

def CalcAccuracyClass(predict_list,file,i):
#     Read training Label
    df = pd.read_csv(file, names = ['labelId'])
    df.index = df.index+1
    df['predicted'] = pd.Series(predict_list, index = df.index)
    df = df[df['labelId']==i]
    match = df[df['labelId'] == df['predicted']]
    predicted_correct = match.shape[0]
    return (predicted_correct/df.shape[0])

def GroupAccuracy(predict_list_train,file):
    for i in range(1,21):
        accuracy_train_BE = CalcAccuracyClass(predict_list_train,file,i)
        print("Group:" + str(i) +" " + str(accuracy_train_BE))
        
def PrintConfusionMatrix(predict_list,file):
    df = pd.read_csv(file, names = ['labelId'])
    df.index = df.index+1
    df['predicted'] = pd.Series(predict_list, index = df.index)
    a = [[0]*21]*21
    for j in range(1,21):
        df_temp = df[df['labelId'] == j]
        for i in range(1,21):
            match = df_temp[df_temp['predicted'] == i]
            a[j][i] = int(match.shape[0])
        print(a[j][1:])


# 2.2.1 Performance on Training Data

In [18]:
predict_list_train = PredictMLE(train_data)
accuracy_train_MLE = CalcAccuracy(predict_list_train,'train_label.csv')
print("MLE Accuracy Train: "+ str(accuracy_train_MLE))


MLE Accuracy Train: 0.991214837163901


In [19]:
GroupAccuracy(predict_list_train,'train_label.csv')

Group:1 0.9979166666666667
Group:2 0.9793459552495697
Group:3 0.9912587412587412
Group:4 0.9880749574105622
Group:5 0.9895652173913043
Group:6 0.9847972972972973
Group:7 0.993127147766323
Group:8 0.9915540540540541
Group:9 0.9966442953020134
Group:10 0.9932659932659933
Group:11 0.9899665551839465
Group:12 1.0
Group:13 0.9898477157360406
Group:14 0.9966329966329966
Group:15 0.9966273187183811
Group:16 0.988313856427379
Group:17 0.9963302752293578
Group:18 0.9911347517730497
Group:19 0.9870689655172413
Group:20 0.9787234042553191


In [20]:
PrintConfusionMatrix(predict_list_train,'train_label.csv')


[479, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 569, 2, 4, 1, 1, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 4, 567, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 2, 1, 580, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 569, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0]
[0, 5, 2, 0, 0, 583, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 578, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 1, 587, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 1, 0, 594, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 590, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 2, 0, 1, 592, 1, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 594, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 2, 0, 2, 0, 0, 0, 0, 0, 585, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 592, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 591, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 592, 1, 0

In [21]:
predict_list_train = PredictBE(train_data)
accuracy_train_BE = CalcAccuracy(predict_list_train,'train_label.csv')
print("BE Accuracy Train: "+ str(accuracy_train_BE))


BE Accuracy Train: 0.941077291685154


In [22]:
GroupAccuracy(predict_list_train,'train_label.csv')


Group:1 0.9666666666666667
Group:2 0.919104991394148
Group:3 0.8793706293706294
Group:4 0.9301533219761499
Group:5 0.9408695652173913
Group:6 0.9493243243243243
Group:7 0.7749140893470791
Group:8 0.9662162162162162
Group:9 0.9630872483221476
Group:10 0.9713804713804713
Group:11 0.9782608695652174
Group:12 0.9797979797979798
Group:13 0.9238578680203046
Group:14 0.9764309764309764
Group:15 0.9780775716694773
Group:16 0.9833055091819699
Group:17 0.9853211009174312
Group:18 0.9680851063829787
Group:19 0.9698275862068966
Group:20 0.7606382978723404


In [23]:
PrintConfusionMatrix(predict_list_train,'train_label.csv')

[464, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 11, 0, 1, 1, 2]
[1, 534, 6, 15, 1, 9, 2, 0, 1, 0, 0, 2, 1, 1, 2, 4, 0, 0, 2, 0]
[1, 10, 503, 23, 1, 20, 2, 0, 0, 0, 0, 7, 1, 1, 0, 2, 0, 0, 1, 0]
[0, 10, 4, 546, 4, 4, 6, 2, 0, 0, 0, 0, 3, 0, 1, 2, 0, 2, 2, 1]
[2, 5, 2, 7, 541, 3, 1, 0, 2, 0, 0, 2, 1, 2, 2, 3, 0, 1, 1, 0]
[0, 11, 8, 1, 2, 562, 0, 0, 1, 1, 0, 2, 0, 1, 1, 0, 1, 0, 1, 0]
[2, 3, 2, 34, 6, 2, 451, 17, 1, 3, 3, 16, 15, 5, 4, 5, 5, 1, 7, 0]
[1, 0, 0, 3, 1, 2, 3, 572, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 3, 1]
[0, 1, 0, 1, 1, 0, 4, 1, 574, 0, 0, 0, 0, 2, 0, 2, 6, 1, 3, 0]
[0, 3, 0, 1, 0, 1, 1, 3, 0, 577, 4, 0, 0, 1, 0, 1, 2, 0, 0, 0]
[1, 0, 1, 2, 0, 1, 0, 2, 0, 0, 585, 1, 0, 0, 0, 1, 0, 2, 2, 0]
[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 582, 0, 1, 0, 0, 3, 1, 5, 0]
[1, 4, 0, 15, 5, 0, 3, 2, 0, 0, 1, 5, 546, 2, 2, 1, 2, 0, 2, 0]
[0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 2, 580, 0, 5, 2, 0, 1, 0]
[2, 2, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 2, 580, 1, 0, 0, 2, 0]
[0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 

# 2.2.2 Performance on Testing Data

In [24]:
test_data = pd.read_csv('test_data.csv', names = ['docIdx', 'wordIdx', 'count'])
test_data.index = test_data.index+1


# MLE 

In [25]:
predict_list_test = PredictMLE(test_data)
accuracy_test_MLE = CalcAccuracy(predict_list_test,'test_label.csv')
print("MLE Accuracy Test: "+ str(accuracy_test_MLE))


MLE Accuracy Test: 0.09460359760159893


In [26]:
GroupAccuracy(predict_list_test,'test_label.csv')

Group:1 0.9937106918238994
Group:2 0.06683804627249357
Group:3 0.04859335038363171
Group:4 0.07142857142857142
Group:5 0.057441253263707574
Group:6 0.08461538461538462
Group:7 0.12041884816753927
Group:8 0.04810126582278481
Group:9 0.05037783375314862
Group:10 0.05289672544080604
Group:11 0.09022556390977443
Group:12 0.043037974683544304
Group:13 0.020356234096692113
Group:14 0.035623409669211195
Group:15 0.04336734693877551
Group:16 0.0678391959798995
Group:17 0.03296703296703297
Group:18 0.0398936170212766
Group:19 0.025806451612903226
Group:20 0.02390438247011952


In [27]:
PrintConfusionMatrix(predict_list_test,'test_label.csv')

[316, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[352, 26, 2, 1, 1, 4, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0]
[354, 3, 19, 6, 3, 1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 1, 0, 0, 0]
[350, 3, 5, 28, 2, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[351, 2, 0, 2, 22, 0, 1, 0, 0, 0, 0, 0, 4, 1, 0, 0, 0, 0, 0, 0]
[351, 2, 0, 1, 1, 33, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[317, 4, 2, 4, 1, 0, 46, 2, 2, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1]
[369, 0, 0, 0, 0, 0, 1, 19, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1]
[376, 0, 0, 0, 0, 0, 0, 1, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[372, 0, 0, 0, 0, 0, 1, 0, 0, 21, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[363, 0, 0, 0, 0, 0, 0, 0, 0, 0, 36, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[375, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 17, 1, 0, 0, 0, 0, 0, 0, 0]
[373, 1, 0, 1, 0, 1, 3, 1, 0, 0, 0, 3, 8, 1, 1, 0, 0, 0, 0, 0]
[375, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 14, 0, 0, 1, 0, 0, 0]
[375, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 0, 0, 0, 0, 0]
[367, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# BE

In [28]:
predict_list_test = PredictBE(test_data)
accuracy_test_BE = CalcAccuracy(predict_list_test,'test_label.csv')
print("BE Accuracy Test: "+ str(accuracy_test_BE))


BE Accuracy Test: 0.7810792804796802


In [29]:
GroupAccuracy(predict_list_test,'test_label.csv')

Group:1 0.7389937106918238
Group:2 0.7609254498714653
Group:3 0.5294117647058824
Group:4 0.7780612244897959
Group:5 0.7127937336814621
Group:6 0.7846153846153846
Group:7 0.5916230366492147
Group:8 0.9012658227848102
Group:9 0.889168765743073
Group:10 0.8690176322418136
Group:11 0.9548872180451128
Group:12 0.9139240506329114
Group:13 0.6590330788804071
Group:14 0.8244274809160306
Group:15 0.8545918367346939
Group:16 0.9472361809045227
Group:17 0.8928571428571429
Group:18 0.8643617021276596
Group:19 0.5935483870967742
Group:20 0.3545816733067729


In [30]:
PrintConfusionMatrix(predict_list_test,'test_label.csv')

[235, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 2, 3, 45, 3, 10, 7, 9]
[3, 296, 6, 12, 7, 22, 1, 3, 2, 0, 0, 17, 4, 4, 7, 4, 0, 0, 1, 0]
[3, 33, 207, 58, 11, 31, 0, 2, 2, 2, 1, 17, 1, 4, 4, 5, 0, 0, 9, 1]
[0, 8, 15, 305, 21, 2, 4, 6, 0, 0, 1, 6, 23, 0, 1, 0, 0, 0, 0, 0]
[0, 8, 10, 37, 273, 3, 4, 4, 1, 1, 0, 6, 17, 8, 2, 0, 3, 0, 6, 0]
[0, 42, 7, 10, 2, 306, 1, 0, 2, 1, 0, 10, 0, 0, 3, 2, 1, 1, 2, 0]
[0, 8, 4, 50, 20, 1, 226, 33, 5, 0, 1, 3, 11, 2, 3, 4, 2, 3, 6, 0]
[1, 1, 0, 2, 0, 1, 5, 356, 4, 2, 0, 1, 4, 0, 2, 1, 4, 2, 9, 0]
[0, 1, 0, 0, 0, 0, 0, 26, 353, 2, 0, 1, 1, 1, 0, 1, 4, 2, 5, 0]
[4, 1, 0, 1, 1, 2, 3, 3, 1, 345, 17, 2, 2, 0, 0, 3, 1, 2, 9, 0]
[2, 0, 0, 0, 0, 0, 1, 1, 0, 4, 381, 1, 0, 2, 1, 2, 0, 1, 3, 0]
[0, 4, 1, 1, 2, 1, 1, 0, 0, 0, 0, 361, 3, 2, 0, 2, 8, 0, 8, 1]
[2, 18, 0, 27, 8, 3, 1, 10, 2, 0, 0, 46, 259, 6, 3, 6, 0, 2, 0, 0]
[10, 7, 1, 3, 0, 0, 0, 4, 0, 1, 0, 1, 3, 324, 3, 17, 3, 6, 10, 0]
[3, 7, 0, 0, 0, 2, 0, 0, 1, 0, 1, 4, 4, 4, 335, 5, 1, 2, 22, 1]
[7, 2, 1, 0, 1, 2, 0, 

# Observations:



### On the train data:
- We find that Maximum Likelihood Works well for the training data and has a high accuracy of approximately 99.1%. 
- We observe that even the Bayesian Estimate works pretty well with an accuracy of approximately 94.1%
- On the training data, the Maximum Likelihood Estimate works better than the Bayesian Estimate.
- Based on the class accuracy, we observe that MLE does exceptionally well for Group 12 and decently well for other cases with an accuracy around 98%
- Based on the class accuracy values, we observe that BE does decently well on most cases with an accuracy of more than 90%(approx). However, it does poorly for Group7 and Group 20 with values around 75%, and Group 3 with an accuracy of 87% approx.
- Based on the confusion matrix, we observe that MLE has very few values that do not lie on the diagonal of the matrix which supports the reason why the training accuracy is around 99%.
- The confusion matrix of BE is densly populated along the major diagonal. However, there are a few inconsistencies but the general trend is along the diagonal which explains why the accuracy is around 94% and not 99%

### On the test data:
- We find that the MLE performs poorly with an accuracy of around 9.5%.
- We find that the Bayesian Estimate performs better than MLE with an accuracy of 78%.
- Although MLE performs better on the training dataset, BE performs better on the test dataset. This means that the Bayesian Estimate is a better generalization and MLE has overfitting issues.
- Based on the class accuracy values, we observe that for BE, there are many values with around 90%(approx) accuracy and a with few around 60%(approx) accuracy. It performs poorly for Group 20 with an accuracy of 35% approximately. There is uniform classification accuracy spread for most groups other than group 20.
- Based on the class accuracy values, we observe that MLE performs extremely well with a 99% accuracy for the first Group and poorly for every other case. This means that the only Group that MLE can classify is Group 1 and this might be related to the fact that MLE classifies most documents as Group 1.
- Based on the Confusion Matrix we see that the first column of MLE's confusion matrix is densely filled with values and this explains why the class accuracy of Group 1 is high. It is not because MLE classifies group 1 documents correctly but  because it classifies most documents as group 1. The MLE generalizes poorly and is a very bad model for this problem.
- The confusion matrix of BE is densely filled on the major diagonal but it still has many inconsistencies with other values filled. This explains why it is not highly extremely accurate(99%) but it still performs way better than MLE.
- One more reason why it does not achieve very high accuracy can be related to the fact that the train dataset does not span the vocabulary data and there are some values that are encountered in the training set for the first time.

### Conclusion:
- Although MLE performs better than BE on the training dataset, it does not generalize well and performs poorly on the training dataset. Hence, the Bayesian Estimate is better than Maximum Likelihood Estimate to model this problem.


# Summarized Results:
    

In [31]:
# Class Priors:
# P(omega = 1) : 0.04259472890229834
# P(omega = 2) : 0.05155736977549028
# P(omega = 3) : 0.05075871860857219
# P(omega = 4) : 0.05208980388676901
# P(omega = 5) : 0.051024935664211554
# P(omega = 6) : 0.052533498979501284
# P(omega = 7) : 0.051646108794036735
# P(omega = 8) : 0.052533498979501284
# P(omega = 9) : 0.052888455053687104
# P(omega = 10) : 0.0527109770165942
# P(omega = 11) : 0.05306593309078002
# P(omega = 12) : 0.0527109770165942
# P(omega = 13) : 0.05244475996095483
# P(omega = 14) : 0.0527109770165942
# P(omega = 15) : 0.052622237998047744
# P(omega = 16) : 0.05315467210932647
# P(omega = 17) : 0.04836276510781791
# P(omega = 18) : 0.05004880646020055
# P(omega = 19) : 0.04117490460555506
# P(omega = 20) : 0.033365870973467035

# Performance On Training Data:

# MLE

In [32]:
# Overall Accuracy: 0.991214837163901

# Class Accuracy:  

# Group:1 0.9979166666666667
# Group:2 0.9793459552495697
# Group:3 0.9912587412587412
# Group:4 0.9880749574105622
# Group:5 0.9895652173913043
# Group:6 0.9847972972972973
# Group:7 0.993127147766323
# Group:8 0.9915540540540541
# Group:9 0.9966442953020134
# Group:10 0.9932659932659933
# Group:11 0.9899665551839465
# Group:12 1.0
# Group:13 0.9898477157360406
# Group:14 0.9966329966329966
# Group:15 0.9966273187183811
# Group:16 0.988313856427379
# Group:17 0.9963302752293578
# Group:18 0.9911347517730497
# Group:19 0.9870689655172413
# Group:20 0.9787234042553191

# Confusion Matrix:

# [479, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
# [0, 569, 2, 4, 1, 1, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# [0, 4, 567, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# [0, 2, 1, 580, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# [1, 0, 0, 1, 569, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0]
# [0, 5, 2, 0, 0, 583, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
# [0, 0, 0, 0, 0, 1, 578, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
# [0, 0, 0, 1, 0, 0, 1, 587, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# [0, 0, 0, 0, 1, 0, 1, 0, 594, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# [0, 0, 0, 0, 0, 0, 1, 0, 0, 590, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# [0, 0, 0, 1, 0, 0, 0, 2, 0, 1, 592, 1, 0, 0, 0, 1, 0, 0, 0, 0]
# [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 594, 0, 0, 0, 0, 0, 0, 0, 0]
# [0, 0, 0, 1, 2, 0, 2, 0, 0, 0, 0, 0, 585, 1, 0, 0, 0, 0, 0, 0]
# [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 592, 0, 0, 0, 0, 0, 0]
# [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 591, 0, 0, 0, 0, 0]
# [1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 592, 1, 0, 0, 3]
# [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 543, 0, 0, 0]
# [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 2, 0, 559, 1, 0]
# [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 458, 1]
# [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 1, 0, 1, 368]

# BE

In [33]:
# Overall Accuracy: 0.941077291685154

# Class Accuracy:

# Group:1 0.9666666666666667
# Group:2 0.919104991394148
# Group:3 0.8793706293706294
# Group:4 0.9301533219761499
# Group:5 0.9408695652173913
# Group:6 0.9493243243243243
# Group:7 0.7749140893470791
# Group:8 0.9662162162162162
# Group:9 0.9630872483221476
# Group:10 0.9713804713804713
# Group:11 0.9782608695652174
# Group:12 0.9797979797979798
# Group:13 0.9238578680203046
# Group:14 0.9764309764309764
# Group:15 0.9780775716694773
# Group:16 0.9833055091819699
# Group:17 0.9853211009174312
# Group:18 0.9680851063829787
# Group:19 0.9698275862068966
# Group:20 0.7606382978723404
    
# Confusion Matrix:
# [464, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 11, 0, 1, 1, 2]
# [1, 534, 6, 15, 1, 9, 2, 0, 1, 0, 0, 2, 1, 1, 2, 4, 0, 0, 2, 0]
# [1, 10, 503, 23, 1, 20, 2, 0, 0, 0, 0, 7, 1, 1, 0, 2, 0, 0, 1, 0]
# [0, 10, 4, 546, 4, 4, 6, 2, 0, 0, 0, 0, 3, 0, 1, 2, 0, 2, 2, 1]
# [2, 5, 2, 7, 541, 3, 1, 0, 2, 0, 0, 2, 1, 2, 2, 3, 0, 1, 1, 0]
# [0, 11, 8, 1, 2, 562, 0, 0, 1, 1, 0, 2, 0, 1, 1, 0, 1, 0, 1, 0]
# [2, 3, 2, 34, 6, 2, 451, 17, 1, 3, 3, 16, 15, 5, 4, 5, 5, 1, 7, 0]
# [1, 0, 0, 3, 1, 2, 3, 572, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 3, 1]
# [0, 1, 0, 1, 1, 0, 4, 1, 574, 0, 0, 0, 0, 2, 0, 2, 6, 1, 3, 0]
# [0, 3, 0, 1, 0, 1, 1, 3, 0, 577, 4, 0, 0, 1, 0, 1, 2, 0, 0, 0]
# [1, 0, 1, 2, 0, 1, 0, 2, 0, 0, 585, 1, 0, 0, 0, 1, 0, 2, 2, 0]
# [0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 582, 0, 1, 0, 0, 3, 1, 5, 0]
# [1, 4, 0, 15, 5, 0, 3, 2, 0, 0, 1, 5, 546, 2, 2, 1, 2, 0, 2, 0]
# [0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 2, 580, 0, 5, 2, 0, 1, 0]
# [2, 2, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 2, 580, 1, 0, 0, 2, 0]
# [0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 589, 1, 3, 2, 0]
# [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 537, 2, 3, 0]
# [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 2, 0, 6, 0, 546, 5, 0]
# [2, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 1, 0, 1, 2, 2, 450, 0]
# [25, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 39, 15, 4, 4, 286]

# Performance on Test Data:


# MLE

In [34]:
# Overall Accuracy: 0.09460359760159893

# Class Accuracy:
# Group:1 0.9937106918238994
# Group:2 0.06683804627249357
# Group:3 0.04859335038363171
# Group:4 0.07142857142857142
# Group:5 0.057441253263707574
# Group:6 0.08461538461538462
# Group:7 0.12041884816753927
# Group:8 0.04810126582278481
# Group:9 0.05037783375314862
# Group:10 0.05289672544080604
# Group:11 0.09022556390977443
# Group:12 0.043037974683544304
# Group:13 0.020356234096692113
# Group:14 0.035623409669211195
# Group:15 0.04336734693877551
# Group:16 0.0678391959798995
# Group:17 0.03296703296703297
# Group:18 0.0398936170212766
# Group:19 0.025806451612903226
# Group:20 0.02390438247011952


# Confusion Matrix:
# [316, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
# [352, 26, 2, 1, 1, 4, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0]
# [354, 3, 19, 6, 3, 1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 0, 1, 0, 0, 0]
# [350, 3, 5, 28, 2, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# [351, 2, 0, 2, 22, 0, 1, 0, 0, 0, 0, 0, 4, 1, 0, 0, 0, 0, 0, 0]
# [351, 2, 0, 1, 1, 33, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
# [317, 4, 2, 4, 1, 0, 46, 2, 2, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1]
# [369, 0, 0, 0, 0, 0, 1, 19, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 1]
# [376, 0, 0, 0, 0, 0, 0, 1, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# [372, 0, 0, 0, 0, 0, 1, 0, 0, 21, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# [363, 0, 0, 0, 0, 0, 0, 0, 0, 0, 36, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# [375, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 17, 1, 0, 0, 0, 0, 0, 0, 0]
# [373, 1, 0, 1, 0, 1, 3, 1, 0, 0, 0, 3, 8, 1, 1, 0, 0, 0, 0, 0]
# [375, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 14, 0, 0, 1, 0, 0, 0]
# [375, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 0, 0, 0, 0, 0]
# [367, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 27, 0, 1, 1, 1]
# [350, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 1, 0, 1]
# [358, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 15, 0, 0]
# [297, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 0, 8, 0]
# [237, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 4, 0, 0, 2, 6]




# BE

In [35]:
# Overall Accuracy:
# 0.7810792804796802

# Class Accuracy:
# Group:1 0.7389937106918238
# Group:2 0.7609254498714653
# Group:3 0.5294117647058824
# Group:4 0.7780612244897959
# Group:5 0.7127937336814621
# Group:6 0.7846153846153846
# Group:7 0.5916230366492147
# Group:8 0.9012658227848102
# Group:9 0.889168765743073
# Group:10 0.8690176322418136
# Group:11 0.9548872180451128
# Group:12 0.9139240506329114
# Group:13 0.6590330788804071
# Group:14 0.8244274809160306
# Group:15 0.8545918367346939
# Group:16 0.9472361809045227
# Group:17 0.8928571428571429
# Group:18 0.8643617021276596
# Group:19 0.5935483870967742
# Group:20 0.3545816733067729

# Confusion Matrix:
# [235, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 2, 3, 45, 3, 10, 7, 9]
# [3, 296, 6, 12, 7, 22, 1, 3, 2, 0, 0, 17, 4, 4, 7, 4, 0, 0, 1, 0]
# [3, 33, 207, 58, 11, 31, 0, 2, 2, 2, 1, 17, 1, 4, 4, 5, 0, 0, 9, 1]
# [0, 8, 15, 305, 21, 2, 4, 6, 0, 0, 1, 6, 23, 0, 1, 0, 0, 0, 0, 0]
# [0, 8, 10, 37, 273, 3, 4, 4, 1, 1, 0, 6, 17, 8, 2, 0, 3, 0, 6, 0]
# [0, 42, 7, 10, 2, 306, 1, 0, 2, 1, 0, 10, 0, 0, 3, 2, 1, 1, 2, 0]
# [0, 8, 4, 50, 20, 1, 226, 33, 5, 0, 1, 3, 11, 2, 3, 4, 2, 3, 6, 0]
# [1, 1, 0, 2, 0, 1, 5, 356, 4, 2, 0, 1, 4, 0, 2, 1, 4, 2, 9, 0]
# [0, 1, 0, 0, 0, 0, 0, 26, 353, 2, 0, 1, 1, 1, 0, 1, 4, 2, 5, 0]
# [4, 1, 0, 1, 1, 2, 3, 3, 1, 345, 17, 2, 2, 0, 0, 3, 1, 2, 9, 0]
# [2, 0, 0, 0, 0, 0, 1, 1, 0, 4, 381, 1, 0, 2, 1, 2, 0, 1, 3, 0]
# [0, 4, 1, 1, 2, 1, 1, 0, 0, 0, 0, 361, 3, 2, 0, 2, 8, 0, 8, 1]
# [2, 18, 0, 27, 8, 3, 1, 10, 2, 0, 0, 46, 259, 6, 3, 6, 0, 2, 0, 0]
# [10, 7, 1, 3, 0, 0, 0, 4, 0, 1, 0, 1, 3, 324, 3, 17, 3, 6, 10, 0]
# [3, 7, 0, 0, 0, 2, 0, 0, 1, 0, 1, 4, 4, 4, 335, 5, 1, 2, 22, 1]
# [7, 2, 1, 0, 1, 2, 0, 0, 0, 0, 0, 1, 0, 1, 0, 377, 2, 2, 1, 1]
# [1, 0, 0, 0, 1, 0, 1, 2, 1, 1, 1, 3, 0, 1, 2, 3, 325, 2, 16, 4]
# [12, 1, 0, 0, 0, 0, 0, 2, 1, 1, 1, 4, 0, 0, 0, 8, 3, 325, 18, 0]
# [6, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 3, 0, 3, 7, 3, 95, 5, 184, 1]
# [47, 3, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 3, 5, 70, 19, 5, 8, 89]
